In [ ]:
pip install langchain langchain-google-genai langgraph

In [6]:
import os, getpass

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google Gemini API Key: ")


Enter your Google Gemini API Key:  ········


In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, List
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import trim_messages


In [8]:
class State(TypedDict):
    messages: List
    language: str


In [9]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")


In [10]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert career counsellor. 
Provide career guidance, skill recommendations, growth strategies, 
and suggest popular online courses (Coursera, Udemy, edX, LinkedIn Learning).
Always answer in a supportive and structured way."""
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [11]:
# Keep only last 5 messages OR 1000 tokens (whichever comes first)
trimmer = RunnableLambda(
    lambda msgs: trim_messages(
        msgs,
        max_tokens=1000,
        strategy="last",   # keep most recent
        token_counter=len, # simplistic, better to use model tokenizer
        start_on="human",  # trim from first human message
        end_on=None
    )
)


In [12]:
def call_model(state: State):
    # Trim history
    trimmed_messages = trimmer.invoke(state["messages"])

    # Debug log
    # print(f"Messages before trim: {len(state['messages'])}, after trim: {len(trimmed_messages)}")

    # Create prompt
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )

    # Get response
    response = model.invoke(prompt)

    return {"messages": trimmed_messages + [response]}


In [13]:
workflow = StateGraph(State)
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [14]:
config = {"configurable": {"thread_id": "career123"}}

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye, and best of luck in your career journey!")
        break

    input_messages = [HumanMessage(user_input)]

    full_response = []
    for chunk, metadata in app.stream(
        {"messages": input_messages, "language": "English"},
        config,
        stream_mode="messages",
    ):
        if isinstance(chunk, AIMessage):
            full_response.append(chunk.content)
            print(chunk.content, end="", flush=True)
    print("\n")


You:  hi


Messages before trim: 1, after trim: 1
Hi there!  It's great to connect.  To best help you with your career, I need a little more information.  Tell me about yourself and your career goals.  For example:

 career change) current situation?** (e.g., employed, unemployed, student, considering a
* **What's your background?** (e.g., education, work experience, skills)
 and passions?** (What do you enjoy doing, even outside of work?)
 goals?) are your career aspirations?** (What kind of job or career are you aiming for? What are your short-term and long-term
* **What are your concerns or challenges?** (e.g., lack of experience, needing to upskill, unsure of your direction)

 personalized career plan. Let's work together to achieve your career goals!




You:  i m a btech student


Messages before trim: 1, after trim: 1
That's great!  Being a B.Tech student opens up a wide range of exciting career paths. To give you the best career guidance, I need a little more information.  Please tell me:

 Engineering, Electrical Engineering, Civil Engineering, etc.)  This is crucial for tailoring advice to your specific skillset.

 passions?**  What do you enjoy doing, both inside and outside of academics?  A career that aligns with your interests will be far more fulfilling.

)?**  Do you have a specific industry or role in mind?  Are you aiming for a leadership position, entrepreneurship, or something else?

 vital for career planning.  What are you naturally good at, and what areas need improvement?

Once I have this information, I can provide you with personalized recommendations for:

* **Career paths:**  Exploring potential roles that align with your background and aspirations.
 need to acquire or enhance to be competitive in the job market.  This will include both tec

You:  my name is gaurav and im doing computer engg.


...

Messages before trim: 1, after trim: 1
It's great that you're considering your career path!  To give you the best career guidance, I need a little more information.  "What engineering?" is a broad question.  Please tell me:

e.g., Mechanical, Electrical, Chemical, Civil, Computer, Biomedical, etc.) This is crucial for tailoring advice.

 you in your program?** (e.g., Freshman, Sophomore, Junior, Senior, Graduated) This helps me understand your current stage and what's realistically achievable.

  (e.g., design, coding, research, management, sustainability)  Knowing your passions will help us identify the most fulfilling career options.

 title, industry, company size, salary expectations)  Having a clear vision is essential for effective planning.

5. **What are your strengths and weaknesses?** (e.g., problem-solving skills, teamwork, communication, technical proficiency) Self-awareness is key to career success.


 information, I can provide personalized recommendations on skills to de

You:  nah i have already told you that im doing engg. can u tell my branch name?


Messages before trim: 1, after trim: 1
I understand you're studying engineering, but I cannot access your personal information, including your engineering branch.  To give you the best career guidance, I need some information from you.  Please tell me:

, Civil Engineering, Chemical Engineering, etc.)  This is crucial for tailoring advice.neering, Computer Science Engineering

 year, Second year, Final year)  This helps me focus on relevant skills and opportunities.

 your interests will help me suggest more relevant career paths.d in design, research, development, management, or something else?  Knowing

 and responsibilities are you looking for?have a specific career in mind?  If so, what is it? If not, what kind of work environment

5. **Your current skillset:** What technical and soft skills do you already possess?  This helps me identify areas for improvement.


 on platforms like Coursera, Udemy, edX, and LinkedIn Learning.  Let's work together to build a strong career foundation

You:  exit


Bot: Goodbye, and best of luck in your career journey!


In [15]:
# career_bot.py
import os
import getpass
import json
from typing import TypedDict, List, Optional

# ------------- API Key -------------
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass(
        "Enter your Google Gemini API Key (GOOGLE_API_KEY): "
    )

# ------------- Imports -------------
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# ------------- Token counter (try tiktoken, fallback to words) -------------
try:
    import tiktoken

    def token_count_text(text: str) -> int:
        # cl100k_base works reasonably for many models; adjust if you have a Gemini tokenizer
        enc = tiktoken.get_encoding("cl100k_base")
        return len(enc.encode(text))
except Exception:
    def token_count_text(text: str) -> int:
        return max(1, len(text.split()))

# ------------- Config (tune these) -------------
MAX_TOKENS_HISTORY = 900     # approx tokens to keep in history
MAX_MESSAGES_HISTORY = 8     # fallback limit to keep if token counting not perfect
SUMMARY_ROUND_TOKEN_LIMIT = 300  # how long summary we allow (in tokens)

# ------------- State definition -------------
class State(TypedDict):
    messages: List  # list of HumanMessage/AIMessage
    language: str
    summary: Optional[str]  # condensed summary of older conversation

# ------------- Initialize Gemini model -------------
# Choose your model string: "gemini-1.5-flash" or "gemini-1.5-pro" etc.
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# ------------- Prompt templates -------------
# Main career counselor template — it uses {summary} (short user profile) and messages placeholder
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are an expert career counsellor. Use the user summary (if present) to "
                "personalize answers. Provide structured, actionable advice: career suggestions, "
                "skills to learn, growth strategies, and 2-4 recommended online courses (platform + course name). "
                "Be concise and give bullets and next steps."
            ),
        ),
        # optional short user-profile summary
        ("system", "User summary (if any): {summary}"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Summarizer template (to compress removed old chats into a short profile)
summarizer_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a summarization assistant. Read the conversation below and produce "
                "a concise, factual bulleted summary (3-6 bullets) containing: user's background, "
                "career preferences, preferred technologies, constraints (time/location), goals, "
                "and important follow-ups. Output only the bullet list (no commentary)."
            ),
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# ------------- Helper trimming function -------------
def simple_trim(messages, max_tokens=MAX_TOKENS_HISTORY, max_messages=MAX_MESSAGES_HISTORY):
    """
    Keeps the most recent messages while staying under max_tokens or max_messages.
    Returns (kept_messages, removed_messages).
    """
    if not messages:
        return [], []

    rev = list(reversed(messages))
    kept = []
    token_sum = 0
    for m in rev:
        text = (m.content or "") if hasattr(m, "content") else str(m)
        t = token_count_text(text)
        if (len(kept) < max_messages) and (token_sum + t <= max_tokens):
            kept.append(m)
            token_sum += t
        else:
            # stop adding, these and everything older is removed
            break
    kept.reverse()
    removed = messages[: len(messages) - len(kept)]
    return kept, removed

# ------------- Summarizer function -------------
def summarize_removed(removed_messages, existing_summary: Optional[str] = None) -> str:
    """
    Summarize removed messages using the model and append to existing_summary.
    Keeps summaries short.
    """
    if not removed_messages:
        return existing_summary or ""

    # Create a prompt using the summarizer_template
    prompt = summarizer_template.invoke({"messages": removed_messages})
    resp = model.invoke(prompt)

    new_summary_piece = resp.content.strip()
    if existing_summary:
        combined = existing_summary.strip() + "\n" + new_summary_piece
    else:
        combined = new_summary_piece

    # Optionally compress the combined summary if it's long
    # (We do a naive trim by token count: ask the model to compress)
    if token_count_text(combined) > SUMMARY_ROUND_TOKEN_LIMIT:
        compress_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "Compress the following summary to 4 bullet points, preserving key facts and constraints. Output only bullets."
                ),
                MessagesPlaceholder(variable_name="messages"),
            ]
        )
        # wrap combined into a single HumanMessage so template accepts it
        compressed_prompt = compress_prompt.invoke({"messages": [HumanMessage(combined)]})
        compressed_resp = model.invoke(compressed_prompt)
        combined = compressed_resp.content.strip()

    return combined

# ------------- The single graph node that trims, summarizes, and calls the model -------------
def call_model(state: State):
    # state arrives with previous memory merged by MemorySaver plus incoming messages
    messages = state.get("messages", []) or []
    language = state.get("language", "English")
    existing_summary = state.get("summary", "") or ""

    # 1) Trim
    kept_messages, removed_messages = simple_trim(messages)
    # debugging prints (optional)
    # print(f"[debug] before={len(messages)} kept={len(kept_messages)} removed={len(removed_messages)}")

    # 2) If some messages were removed, summarize them and update summary
    if removed_messages:
        new_summary = summarize_removed(removed_messages, existing_summary)
    else:
        new_summary = existing_summary

    # 3) Build final prompt and call the model.
    #    prompt_template expects {summary} and messages placeholder
    prompt = prompt_template.invoke({"messages": kept_messages, "summary": new_summary, "language": language})
    response = model.invoke(prompt)

    # 4) Return updated state: keep the trimmed history + the new response, and store updated summary
    new_messages = kept_messages + [response]
    return {"messages": new_messages, "language": language, "summary": new_summary}

# ------------- Build workflow and memory -------------
workflow = StateGraph(state_schema=State)
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

memory = MemorySaver()  # uses persistent storage internally (depending on langgraph version)
app = workflow.compile(checkpointer=memory)

# ------------- Simple CLI loop with streaming -------------
def chat_loop():
    thread_id = "career_thread_001"
    config = {"configurable": {"thread_id": thread_id}}

    print("CareerBot — type 'exit' to quit.")
    while True:
        user_input = input("You: ").strip()
        if not user_input:
            continue
        if user_input.lower() in ("exit", "quit"):
            print("Bot: Goodbye — good luck on your career journey!")
            break

        input_messages = [HumanMessage(user_input)]
        # stream response pieces
        full = []
        for chunk, metadata in app.stream(
            {"messages": input_messages, "language": "English"},
            config,
            stream_mode="messages",
        ):
            if isinstance(chunk, AIMessage):
                # print streaming chunk
                print(chunk.content, end="", flush=True)
                full.append(chunk.content)
        print("\n")  # newline after message completes

if __name__ == "__main__":
    chat_loop()



CareerBot — type 'exit' to quit.


You:  hi


[debug] before=1 kept=1 removed=0
Hi there! To give you the best career advice, I need some information from you.  Please tell me about:

 your current job (if any)? What is your education level? What are your interests and passions?
 you envision yourself doing? What are your long-term aspirations? What is your ideal work-life balance?
 you good at? What do you enjoy doing?  What skills do you already possess?
* **Your challenges:** What are your obstacles or concerns about your career?  Are there any skills you feel you lack?

 this information, I can provide you with personalized and actionable career advice, including potential career paths, skills to develop, and recommended online courses.




You:  im a btech student


[debug] before=1 kept=1 removed=0
Okay, you're a B.Tech student.  To give you the best career advice, I need more information.  However, I can offer some general guidance and actionable steps based on common B.Tech career paths.  Please tell me:

, Mechanical Engineering, Electrical Engineering, etc.)?**  This significantly impacts career options.
g., problem-solving, creativity, teamwork, coding, design, etc.)
* **What kind of work environment do you prefer?** (e.g., fast-paced, collaborative, independent, etc.)
 are your long-term career goals?** (e.g., entrepreneurship, management, research, etc.)


Until I have this information, here's some general advice applicable to many B.Tech graduates:


 Suggestions:**r

* **Software Engineering/Development:** High demand, diverse roles, potential for high earning.
* **Data Science/Analytics:** Growing field with strong job prospects. Requires strong mathematical and programming skills.
* **Project Management:**  Involves planning, execution

...

[debug] before=1 kept=1 removed=0
Hi Gaurav,

 your background, interests, and goals.  Please tell me about:bout

 **Your current situation:** What is your current job (if any)? What is your education level? What are your skills?
 do you enjoy doing? What are you passionate about? What kind of work environment do you prefer?
 you aiming for? What are your short-term and long-term career aspirations?  What is your ideal salary and work-life balance?

 and educational resources.  Let's work together to build a successful career plan for you.skills development,




You:  what is my name?


[debug] before=1 kept=1 removed=0
I do not have access to your personal information, including your name.  To help you with your career, I need some information from you. Please tell me about:

* **Your interests:** What subjects or activities do you enjoy?
 soft skills, creative skills, etc.) at?  (Technical skills,
* **Your experience:** What jobs have you held? What did you like and dislike about them?
Your education:** What is your highest level of education?
* **Your career goals:** What kind of work do you want to do? What are your long-term aspirations?

 advice, including potential career paths, skills to develop, and recommended online courses.




You:  exit


Bot: Goodbye — good luck on your career journey!
